In [13]:
import sys
sys.path.append(r'd:\\Programming\\Python\\Skripsi\\main\\scripts\\action')

import os

import pandas as pd
import json

import data_prep

### Prepare the destination directory

In [11]:
label = ["normal", "glaukoma"]
dataset_names = ["rimone", 'g1020', 'refuge', 'papila']

path_dataset = r'd://Programming/Python/Skripsi/dataset/'
path_rimone = os.path.join(path_dataset, r'RIM-ONE_DL_images/partitioned_by_hospital')
path_g1020 = os.path.join(path_dataset, r'G1020/Images')
path_refuge = os.path.join(path_dataset, r'REFUGE/Images_Square')
path_papila = os.path.join(path_dataset, r'PAPILA/FundusImages')

path_dataset_destination = r'd://Programming/Python/Skripsi/dataset/combined'

In [3]:
data_prep.create_destination_directory(path=path_dataset_destination,
                                       dataset_names=dataset_names,
                                       label_names=label)

{'Success': [],
 'Already Exists': ['d://Programming/Python/Skripsi/dataset/combined\\rimone\\normal',
  'd://Programming/Python/Skripsi/dataset/combined\\rimone\\glaukoma',
  'd://Programming/Python/Skripsi/dataset/combined\\g1020\\normal',
  'd://Programming/Python/Skripsi/dataset/combined\\g1020\\glaukoma',
  'd://Programming/Python/Skripsi/dataset/combined\\refuge\\normal',
  'd://Programming/Python/Skripsi/dataset/combined\\refuge\\glaukoma',
  'd://Programming/Python/Skripsi/dataset/combined\\papila\\normal',
  'd://Programming/Python/Skripsi/dataset/combined\\papila\\glaukoma']}

## RIM-ONE Dataset
### Getting the file name

In [4]:
file_names_raw_rimone = {
    "training_set": {
        "normal": [],
        "glaucoma": []
    },
    "test_set": {
        "normal": [],
        "glaucoma": []
    }
}

In [5]:
for dataset_name_key, dataset_name_value in file_names_raw_rimone.items():
    for label_name_key, label_name_value in dataset_name_value.items():
        file_names_raw_rimone[dataset_name_key][label_name_key] = data_prep.get_file_names(path=os.path.join(path_rimone,
                                                                                                             dataset_name_key,
                                                                                                             label_name_key))

### Validate the file count

In [6]:
for dataset_name_key, dataset_name_value in file_names_raw_rimone.items():
    for label_name_key, label_name_value in dataset_name_value.items():
        print(f"{dataset_name_key[:5]} {label_name_key}",
              f"Total: {len(label_name_value)}",
              sep="\n")

train normal
Total: 195
train glaucoma
Total: 116
test_ normal
Total: 118
test_ glaucoma
Total: 56


### Transform the raw data into cleaned one

In [7]:
file_names_cleaned_rimone = {
    label[0]: [],
    label[1]: []
}
for data_type_key, data_type_value in file_names_raw_rimone.items():
    for file_name_key, file_name_value in data_type_value.items():
        if file_name_key == "normal":
            file_names_cleaned_rimone[label[0]].extend(file_name_value)
        elif file_name_key == "glaucoma":
            file_names_cleaned_rimone[label[1]].extend(file_name_value)

### Check the validity

In [8]:
for label_name, label_file in file_names_cleaned_rimone.items():
    print(f"Total {label_name} : {len(label_file)}")

Total normal : 313
Total glaukoma : 172


### Copying the image

In [9]:
for dataset_name_key, dataset_name_value in file_names_raw_rimone.items():
    for label_name_key, label_name_value in dataset_name_value.items():
        if label_name_key == "normal":
            data_prep.copy_files(source_path=os.path.join(path_rimone,
                                                        dataset_name_key,
                                                        label_name_key),
                                destination_path=os.path.join(path_dataset_destination,
                                                            dataset_names[0],
                                                            label[0]),
                                file_names=label_name_value)
        elif label_name_key == "glaucoma":
            data_prep.copy_files(source_path=os.path.join(path_rimone,
                                                        dataset_name_key,
                                                        label_name_key),
                                destination_path=os.path.join(path_dataset_destination,
                                                            dataset_names[0],
                                                            label[1]),
                                file_names=label_name_value)

### Validate the file

In [12]:
for label_name in label:
    for file_name in data_prep.get_file_names(path=os.path.join(path_dataset_destination,
                                                                 dataset_names[0])):
        if file_name in file_names_cleaned_rimone[label_name]:
            pass
        else:
            print(f"{file_name} not found in {label_name}")

## G1020 Dataset
### Getting the file name

In [17]:
file_names_raw_g1020 = {
    label[0]: [],
    label[1]: []
}
metadata_g1020 = pd.read_csv(os.path.join(path_g1020, "../G1020.csv"))

In [18]:
metadata_g1020.head()

,imageID,binaryLabels
0,image_0.jpg,0
1,image_1.jpg,0
2,image_3.jpg,0
3,image_4.jpg,0
4,image_5.jpg,0


In [19]:
metadata_g1020.binaryLabels.value_counts()

binaryLabels
0    724
1    296
Name: count, dtype: int64

In [20]:
metadata_g1020.loc[metadata_g1020.binaryLabels == 0, "binaryLabels"] = label[0]
metadata_g1020.loc[metadata_g1020.binaryLabels == 1, "binaryLabels"] = label[1]
metadata_g1020.binaryLabels.value_counts()

C:\Users\Bugi\AppData\Local\Temp\ipykernel_15664\4072057408.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'normal' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  metadata_g1020.loc[metadata_g1020.binaryLabels == 0, "binaryLabels"] = label[0]


binaryLabels
normal      724
glaukoma    296
Name: count, dtype: int64

In [29]:
metadata_g1020.rename(columns={"imageID":"file name",
                               "binaryLabels":"label"},
                      inplace=True)
metadata_g1020.head()

,file name,label
0,image_0.jpg,normal
1,image_1.jpg,normal
2,image_3.jpg,normal
3,image_4.jpg,normal
4,image_5.jpg,normal


In [33]:
for row in metadata_g1020.iterrows():
    if row[1]['label'] == label[0]:
        file_names_raw_g1020[label[0]].append(row[1]['file name'])
    elif row[1]['label'] == label[1]:
        file_names_raw_g1020[label[1]].append(row[1]['file name'])

### Validate the file count

In [34]:
for key in file_names_raw_g1020.keys():
    print(f"Total {key} : {len(file_names_raw_g1020[key])}")

Total normal : 724
Total glaukoma : 296


### Copying the image

In [36]:
for label_name, label_file in file_names_raw_g1020.items():
    data_prep.copy_files(source_path=path_g1020,
                         destination_path=os.path.join(path_dataset_destination,
                                                     dataset_names[1],
                                                     label_name),
                         file_names=label_file)

### Validate the file

In [37]:
for label_name in label:
    for file_name in data_prep.get_file_names(path=os.path.join(path_dataset_destination,
                                                                 dataset_names[1])):
        if file_name in file_names_raw_g1020[label_name]:
            pass
        else:
            print(f"{file_name} not found in {label_name}")

## REFUGE Dataset
### Getting the file name

### Validate the file count

### Transform the raw data into cleaned one

### Check the validity

### Copying the image

### Validate the file

## PAPILA Dataset
### Getting the file name

### Validate the file count

### Transform the raw data into cleaned one

### Check the validity

### Copying the image

### Validate the file